In [94]:
import pandas as pd
from pathlib import Path

data_path = Path(r"C:\Python\projects\CGPAPrediction\data\another_try\data_with_genders")

sem_1_2025 = r"! Отчет по РК и экзам 1 сем_выгрузка 21.01.2025_анализ итоговых оценок.xlsx"
sem_2_2024 = r"Результаты летней сессии 2023-2024.xlsx"
sem_1_2024 = r"Итоговые оценки_замняя сессия 2023-2024_15.01.2024.xlsx"
sem_2_2023 = r"! ЛЕТНЯЯ СЕССИЯ 2022-2023_ВСЕ.xls"
sem_1_2023 = r"! Зимняя сессия 2022-2023_для подготовки на УС.xls"

file_names = [sem_1_2023, sem_2_2023, sem_1_2024, sem_2_2024, sem_1_2025]

data = {'sem_1_2023': pd.read_excel(data_path / sem_1_2023), 'sem_2_2023': pd.read_excel(data_path / sem_2_2023), 'sem_1_2024': pd.read_excel(data_path / sem_1_2024), 'sem_2_2024': pd.read_excel(data_path / sem_2_2024), 'sem_1_2025': pd.read_excel(data_path / sem_1_2025)}

In [95]:
def clean_str(s):
    if isinstance(s, str):
        return re.sub(r"\s+", " ", s).strip()
    return s  

for df in data.values():
    for col in df.columns:
        if df[col].dtype == object:  
            df[col] = df[col].apply(clean_str)

In [96]:
to_drop = set()
for name, df in data.items():
    for (fio, op, subj), gdf in df.groupby(['ФИО студента', 'ОП/Специальность', 'Дисциплина']):
        if len(gdf) > 1:
            to_drop.add((fio, op))

In [97]:
for name, df in data.items():
    data[name] = df[~df[['ФИО студента', 'ОП/Специальность']].apply(tuple, axis=1).isin(to_drop)]

In [98]:
drop_keys = set()

chek = {}

for name, df in {k: v for k, v in data.items() if 'ID студента' in v.columns}.items():
    for tp, ids in df.groupby(['ФИО студента', 'ОП/Специальность'])['ID студента'].unique().items():
        if len(ids) > 1:
            drop_keys.add(tp)
        else:
            if chek.setdefault(tp, ids[0]) != ids[0]:
                drop_keys.add(tp)

In [99]:
len(drop_keys)

9

In [100]:
for name, df in data.items():
    print(data[name].shape)
    data[name] = df[~df[['ФИО студента', 'ОП/Специальность']].apply(tuple, axis=1).isin(drop_keys)]
    print(data[name].shape)

(33881, 36)
(33839, 36)
(31624, 24)
(31580, 24)
(42751, 25)
(42726, 25)
(37692, 27)
(37638, 27)
(49145, 24)
(49083, 24)


In [101]:
def is_float(i):
    try:
        float(i)
        return True
    except Exception:
        return False

marks_cols = ['РК 1', 'РК 2', 'Рейтинг', 'Экзамен', 'Итоговая оценка']
for name, df in data.items():
    data[name] = df.dropna(subset=marks_cols)
    for col in marks_cols:
        if data[name][col].dtype == "object":
            data[name][col] = data[name][col].apply(lambda i: float(i) if is_float(i) else 0)

C:\Users\Михаил\AppData\Local\Temp\ipykernel_4880\4176655902.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[name][col] = data[name][col].apply(lambda i: float(i) if is_float(i) else 0)


In [102]:
cols = ["ФИО студента", "ОП/Специальность"]

sets = [set(df[cols].itertuples(index=False, name=None)) for df in data.values()]

common_keys = set.intersection(*sets)
len(common_keys)

2423

In [103]:
for name, df in data.items():
    data[name] = df[df[cols].apply(tuple, axis=1).isin(common_keys)]
    print(data[name].shape)

(17148, 36)
(16346, 24)
(14935, 25)
(14236, 27)
(14817, 24)


In [104]:
for fname, df in data.items():
    data[fname] = df.rename(columns={
    'student_gender': 'Пол студента',
    'teacher_gender': 'Пол преподавателя'
})

In [105]:
def merge_staff(col_name, index_cols, dfs):
    source =  next(iter([df for df in dfs.values() if col_name in df.columns]), None)
    if source is None:
        return
    source = source.groupby(index_cols)[col_name].unique().apply(lambda l : l[0] if len(l) == 1 else print(l)).reset_index()
    for name, df in {k: v for k, v in dfs.items() if not col_name in v.columns}.items():
        dfs[name] = df.merge(source, on=index_cols, how='left')

In [106]:
next(iter(data.values())).head()["Экзамен"].max()

np.float64(100.0)

In [107]:
merge_colls = ['Язык обучения', 'Вид финансирования', 'Форма оплаты', 'ID студента']

for col in merge_colls:
    merge_staff(col, ['ФИО студента', 'ОП/Специальность'], data)
    
for df in data.values():
    print(df.shape)

(17148, 40)
(16346, 24)
(14935, 26)
(14236, 28)
(14817, 26)


In [ ]:
next(iter(data.values())).head()["Экзамен"].max()

In [83]:
next(iter(data.values())).head()["Экзамен"].max()

np.float64(100.0)

In [ ]:
for df in data.values():
    print(df.shape)

In [ ]:
output_path = Path(r"C:\Python\projects\CGPAPrediction\data\another_try\data_with_genders\cleaned_data")

res_cols = ['ФИО студента', 'ОП/Специальность', 'Форма оплаты', 'Вид финансирования', 'Язык обучения', 
            'Институт', 'Форма обучения', 'Группа', 'Дисциплина', 'Семестр', 'ФИО преподавателя', 'Кафедра преподавателя',
            'Пол студента', 'Пол преподавателя', 'РК 1', 'РК 2', 'Рейтинг', 'Экзамен', 'Итоговая оценка']


for fname, df in data.items():
    df[res_cols].to_excel((output_path / fname).with_suffix('.xlsx'), index=False)

In [108]:
import re

semester_mapping = {k: list(map(int, re.findall(r"\d+", k))) for k in data.keys()}
semester_mapping = {k: v[0] + v[1] * 2 for k, v in semester_mapping.items()}
min_sem = min(semester_mapping.values())
semester_mapping = {k: v - min_sem for k, v in semester_mapping.items()}

semester_mapping    

{'sem_1_2023': 0,
 'sem_2_2023': 1,
 'sem_1_2024': 2,
 'sem_2_2024': 3,
 'sem_1_2025': 4}

In [109]:
for k, df in data.items():
    df["Семестр"] = semester_mapping[k]

In [110]:
res_cols = ['ID студента', 'ФИО студента', 'ОП/Специальность', 'Форма оплаты', 'Вид финансирования', 'Язык обучения',
            'Институт', 'Форма обучения', 'Группа', 'Дисциплина', 'Семестр', 'ФИО преподавателя', 'Кафедра преподавателя',
            'Пол студента', 'Пол преподавателя', 'РК 1', 'РК 2', 'Рейтинг', 'Экзамен', 'Итоговая оценка']

for k, df in data.items():
    data[k] = df[res_cols]

In [111]:
result = pd.concat(data.values(), ignore_index=True)

In [112]:
for t, s in result.groupby(['ФИО студента', 'ОП/Специальность'])['Пол студента'].unique().items():
    if len(s) > 1:
        result.loc[(result['ФИО студента'] == t[0]) & (result['ОП/Специальность'] == t[1]), 'Пол студента'] = "-"

In [113]:
for fio in result.loc[result['Пол студента'] == '-', 'ФИО студента'].unique():
    g = input(f"{fio}\n")
    g = 'female' if g == '0' else 'male'
    result.loc[result['ФИО студента'] == fio, 'Пол студента'] = g

In [ ]:
wrong = ["Амантай Қымбат Алматқызы"]
for w in wrong:
    result.loc[result["ФИО студента"] == w, "Пол студента"] = 'female'

In [114]:
for fio in [k for k, v in result.groupby("ФИО преподавателя")["Пол преподавателя"].unique().items() if len(v) > 1]:
    g = input(f"{fio}\n")
    g = 'female' if g == '0' else 'male'
    result.loc[result["ФИО преподавателя"] == fio, "Пол преподавателя"] = g

In [115]:
print([k for k, v in result.groupby("ФИО преподавателя")["Пол преподавателя"].unique().items() if len(v) > 1])

[]


In [116]:
result.to_excel(r"C:\Python\projects\CGPAPrediction\data\another_try\data_with_genders\cleaned_data\re_merged.xlsx", index=False)